<a href="https://colab.research.google.com/github/fluxtransport/glmtools/blob/master/examples/basic_read_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before running this notebook, it's helpful to 

`conda install -c conda-forge nb_conda_kernels`

`conda install -c conda-forge ipywidgets`

and set the kernel to the conda environment in which you installed glmtools (typically, `glmval`)

In [1]:
try:
  import glmtools
except:
  !pip install git+https://github.com/deeplycloudy/glmtools.git
  import glmtools
try:
  import siphon
except:
  !pip install siphon
try: 
  import netCDF4 as netcdf
except:
  !pip install netCDF4
  import netCDF4 as netcdf

  Cloning https://github.com/deeplycloudy/glmtools.git to /tmp/pip-req-build-9s6bvu8v
  Running command git clone -q https://github.com/deeplycloudy/glmtools.git /tmp/pip-req-build-9s6bvu8v
  Created wheel for glmtools: filename=glmtools-0.1.dev0-cp36-none-any.whl size=1854547 sha256=4e6f7750ef80b2644094e35003d68501313561c4c7c100998b2dd15475c3066f
  Stored in directory: /tmp/pip-ephem-wheel-cache-siomjjnd/wheels/97/c8/2c/6e3d66d92d669f65ff48f735cc61670efc5c54dbcc18d401fe
Successfully built glmtools
     |████████████████████████████████| 71kB 2.9MB/s 
     |████████████████████████████████| 4.1MB 4.7MB/s 
     |████████████████████████████████| 327kB 44.2MB/s 


In [0]:
import os
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from glmtools.io.glm import GLMDataset

## Use a sample data file included in glmtools

In [0]:

from glmtools.test.common import get_sample_data_path

sample_path = get_sample_data_path()
samples = [
    "OR_GLM-L2-LCFA_G16_s20181830433000_e20181830433200_c20181830433231.nc",
    "OR_GLM-L2-LCFA_G16_s20181830433200_e20181830433400_c20181830433424.nc",
    "OR_GLM-L2-LCFA_G16_s20181830433400_e20181830434000_c20181830434029.nc",
]
samples = [os.path.join(sample_path, s) for s in samples]
filename = samples[0]

## Use data from the most recent minute or two

Requires siphon.

To load data via siphon from opendap, you must

`conda install -c conda-forge siphon`

In [0]:
# Load data from the most recent minute or two!
if False:
    from siphon.catalog import TDSCatalog
    g16url = "http://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GRB16/GLM/LCFA/current/catalog.xml"
    satcat = TDSCatalog(g16url)
    filename = satcat.datasets[-1].access_urls['OPENDAP']

## Load the data

In [0]:
glm = GLMDataset(filename)
print(glm.dataset)

<xarray.Dataset>
Dimensions:                                 (number_of_events: 18361, number_of_field_of_view_bounds: 2, number_of_flashes: 302, number_of_groups: 7182, number_of_time_bounds: 2, number_of_wavelength_bounds: 2)
Coordinates:
    event_id                                (number_of_events) uint32 1120987976 ... 1121006973
    event_time_offset                       (number_of_events) datetime64[ns] ...
    event_lat                               (number_of_events) float32 -32.066833 ... -32.166367
    event_lon                               (number_of_events) float32 -57.755478 ... -56.973434
    event_parent_group_id                   (number_of_events) uint32 488999337 ... 489007084
    group_id                                (number_of_groups) uint32 488999337 ... 489007084
    group_time_offset                       (number_of_groups) datetime64[ns] ...
    group_lat                               (number_of_groups) float32 -32.065823 ... -32.16687
    group_lon        

## Flip through each flash, plotting each.

Event centroids are small black squares.

Group centroids are white circles, colored by group energy.

Flash centroids are red 'x's

In [0]:
from glmtools.plot.locations import plot_flash

import ipywidgets as widgets
# print(widgets.Widget.widget_types.values())
fl_id_vals = list(glm.dataset.flash_id.data)
fl_id_vals.sort()
flash_slider =  widgets.SelectionSlider(
    description='Flash',
    options=fl_id_vals,
)

def do_plot(flash_id):
    fig = plot_flash(glm, flash_id)
widgets.interact(do_plot, flash_id=flash_slider)

interactive(children=(SelectionSlider(description='Flash', options=(44442, 44444, 44446, 44452, 44455, 44456, …

<function __main__.do_plot>

# Find flashes in some location

There are hundreds of flashes to browse above, and they are randomly scattered across the full disk. Storms near Lubbock, TX at the time of sample data file had relatively low flash rates, so let's find those.

In [0]:
flashes_subset = glm.subset_flashes(lon_range = (-102.5, -100.5), lat_range = (32.5, 34.5))
print(flashes_subset)

<xarray.Dataset>
Dimensions:                                 (number_of_events: 18361, number_of_field_of_view_bounds: 2, number_of_flashes: 302, number_of_groups: 7182, number_of_time_bounds: 2, number_of_wavelength_bounds: 2)
Coordinates:
    event_id                                (number_of_events) uint32 1120992784 ... 1120993429
    event_time_offset                       (number_of_events) datetime64[ns] ...
    event_lat                               (number_of_events) float32 34.396645 ... -32.704655
    event_lon                               (number_of_events) float32 -83.24805 ... -54.90966
    event_parent_group_id                   (number_of_events) uint32 489001354 ... 489001578
    group_id                                (number_of_groups) uint32 489001354 ... 489001578
    group_time_offset                       (number_of_groups) datetime64[ns] ...
    group_lat                               (number_of_groups) float32 34.344788 ... -32.703754
    group_lon           

In [0]:
from glmtools.plot.locations import plot_flash

import ipywidgets as widgets
# print(widgets.Widget.widget_types.values())
fl_id_vals = list(flashes_subset.flash_id.data)
fl_id_vals.sort()
flash_slider =  widgets.SelectionSlider(
    description='Flash',
    options=fl_id_vals,
)

# from functools import partial
# glm_plotter = partial(plot_flash, glm) # fails with a __name__ attr not found
def do_plot(flash_id):
    fig = plot_flash(glm, flash_id)
widgets.interact(do_plot, flash_id=flash_slider)

interactive(children=(SelectionSlider(description='Flash', options=(44442, 44444, 44446, 44452, 44455, 44456, …

<function __main__.do_plot>